In [53]:
import sys
import numpy as np
from pyofss import Domain, System, Sech, Fibre
from pyofss.domain import lambda_to_nu, nu_to_lambda
from pyofss import map_plot, labels, single_plot, energy
from matplotlib import pyplot as plt

import matplotlib
from matplotlib import pyplot as plt
<!-- %matplotlib qt5
matplotlib.use('Qt5Agg') -->

In [57]:
def dB(num):
    return 10 * np.log10(num)

domain = Domain(bit_width=10.0, samples_per_bit=2**13,
                centre_nu=lambda_to_nu(835.0))

P_0 = 3480.      # peak power W
width = 0.010   # pulse duration 10 fs
gamma = 110     # (W km)^-1

beta_full = [0.0, 0.0, -11.830, 
           8.1038e-2,  -9.5205e-5,   2.0737e-7,  
          -5.3943e-10,  1.3486e-12, -2.5495e-15, 
           3.0524e-18, -1.7140e-21]


L_d = (width**2)/abs(beta_full[2])
L_nl = 1/(gamma*P_0)

z_sol = np.pi/2*L_d
print(" L_nl={:.2f} cm\n L_d={:.2f} cm\n z_sol={:.2f}cm\n N={:.2f}\n P={:.3f} kW".format(L_nl*1e5, L_d*1e5, z_sol*1e5, np.abs((L_d/L_nl))**0.5, P_0*1e-3))

length = 50*1e-5 # length in km

system = System(domain)
system.add(Sech(peak_power=P_0, width=width, using_fwhm=True))
system.add(Fibre(length=length, gamma=gamma, beta=beta_full, self_steepening=True, use_all='hollenbeck',
                 total_steps=100, traces=100, method='ARK4IP', local_error=0.005))
system.run()

storage = system['fibre'].stepper.storage
(x, y, z_temp) = storage.get_plot_data(False, (lambda_to_nu(1300), lambda_to_nu(500)), True)
z_label = r"Fibre length, $z \, (m)$"
z = z_temp * 1.0e3

fiber = system['fibre']
h_R = fiber.nonlinearity.h_R

map_plot(x, dB(y/np.max(y)), z, labels["nu"], labels["P_nu"], z_label,
         filename="pyofss_fig8_map_nu", y_range=(-40.0, 0))

single_plot(nu_to_lambda(x), y[0], labels["Lambda"], labels["P_lambda"],
         filename="before_fibre_lambda", use_fill=False)

np.savetxt('before_fibre_lambda.dat', np.column_stack(np.array([nu_to_lambda(x), y[0]])), delimiter=' ')

single_plot(nu_to_lambda(x), y[-1], labels["Lambda"], labels["P_lambda"],
         filename="after_fibre_lambda", use_fill=False)

np.savetxt('after_fibre_lambda.dat', np.column_stack(np.array([nu_to_lambda(x),  y[-1]])), delimiter=' ')

 L_nl=0.26 cm
 L_d=0.85 cm
 z_sol=1.33cm
 N=1.80
 P=3.480 kW
Using general expression for nonlinearity


c:\Users\Karin\PycharmProjects\pyofss-master\pyofss\modules\sech.py:137: RuntimeWarning: overflow encountered in cosh
  magnitude = sqrt(self.peak_power) / cosh(t_normalised)
c:\Users\Karin\PycharmProjects\pyofss-master\pyofss\modules\storage.py:212: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  As = np.vstack(np.array(f(zs) + 1j * g(zs))


Attempting to reduce storage arrays to specified range...
Required range: [230.60958307692306, 599.584916]
Actual range: [230.63288383233527, 599.6328838323353]

Generating map_plot...
Wrote file pyofss_fig8_map_nu

Generating single_plot...
Wrote file before_fibre_lambda

Generating single_plot...
Wrote file after_fibre_lambda


In [62]:
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 25,
        }

plt.plot(domain.t - domain.t.min(), ifft(h_R), 'black', linewidth=2)

plt.title("Raman_response, normalised",
          fontsize=25, color='black')
plt.xlabel('t - t_0', fontdict=font)
plt.ylabel('a. u.', fontdict=font)

plt.show()

single_plot(domain.t - domain.t.min(), ifft(h_R), "t - t_0 [pc]", "Raman response (a. u.)",
         filename="raman_response", use_fill=False)

np.savetxt('raman_response.dat', np.column_stack(np.array([domain.t - domain.t.min(),  ifft(h_R)])), delimiter=' ')


Generating single_plot...
Wrote file raman_response


c:\Users\Karin\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1298: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
c:\Users\Karin\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:1298: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [63]:
(x, y, z_temp) = storage.get_plot_data(True, reduced_range=(-1, 4))
z = z_temp * 1.0e3

map_plot(x, dB(y/np.max(y)), z, labels["t"], labels["P_t"], z_label,
        filename="pyofss_fig8_map_t", y_range=(-40.0, 0))

Attempting to reduce storage arrays to specified range...
Required range: [-1, 4]
Actual range: [-0.999755859375, 4.000244140625]

Generating map_plot...


C:\Users\Karin\AppData\Local\Temp\ipykernel_19032\2681115962.py:2: RuntimeWarning: divide by zero encountered in log10
  return 10 * np.log10(num)


Wrote file pyofss_fig8_map_t


In [64]:
single_plot(x, y[0], labels["t"], labels["P_lambda"],
         filename="before_fibre_t", use_fill=False)

np.savetxt('before_fibre_t.dat', np.column_stack(np.array([x,  y[0]])), delimiter=' ')


Generating single_plot...
Wrote file before_fibre_t


In [65]:
single_plot(x, y[-1], labels["t"], labels["P_lambda"],
         filename="after_fibre_t", use_fill=False)

np.savetxt('after_fibre_t.dat', np.column_stack(np.array([x,  y[-1]])), delimiter=' ')


Generating single_plot...
Wrote file after_fibre_t
